Импортируем общий код

In [1]:
%run NER_common.ipynb

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pytorch_transformers
import seqeval.metrics
from tqdm import tqdm_notebook

import numpy as np
from itertools import chain, islice
from collections import Counter
from collections import defaultdict
from functools import partial

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, DataLoader
import pytorch_lightning as pl
from test_tube import Experiment
import argparse
import os.path

Для проведения эксперимента используются pytorch-lightning и test_tube. <br>
Опишем параметры эксперимента. Можно изменять их значения перед дальнейшим исполнением. Также они сохранены в директории experiment_dir/experiment_name/версия-эксперимента (test_tube делает ее автоинкремент основываясь на содержимом директории) в файле meta_tags.csv. <br>
Файл metrics.csv хранит логи метрик (каждый 100 итераций при обучении, также логируются результаты валидации).

Суть эксперимента:
Мы будем производить fine-tuning (дообучение) претренированной модели BERT (bert-base-cased) под текущую задачу. <br>
Причины выбора модели: малое количество данных повышает необходимость в пре-тренированных представлениях, модели, основанные на BERT, показывают высокие результаты на множестве задач NER, модель чувствительна к регистру и использует Byte-pair encoding для улучшения работы с редкими словами.

Поскольку токенизация у этой модели может расцепить оригинальные токены, в этом случае предсказание для первого субтокена из разбиения считается предсказанием для всего оригинального токена. Также для последующих субтокенов не считается loss.

In [4]:
experiment_config = argparse.Namespace()
experiment_config.basic_model = 'BERT'

experiment_config.replace_urls = True
experiment_config.replace_numbers = False 
experiment_config.split_hashtags = False # оставить False, не реализовано, BertTokenizer сам это сделает
experiment_config.split_nicknames = False # оставить False, не реализовано,  BertTokenizer сам это сделает

experiment_config.val_batch_size = 24
experiment_config.train_batch_size = 16
experiment_config.lr = 2e-5
experiment_config.gradient_acccumulation_steps = 1 # фактически batch_size = train_batch_size * gradient_accumulation_steps
experiment_config.gradient_clipping_norm = 5.0
experiment_config.n_epochs = 5
experiment_config.mask_additional_wordpieces = True # оставить True, не реализовано

experiment_dir = 'NER_experiments/'
experiment_config.experiment_name = 'BERT_finetune_mask'

Прочтём файлы и подсчитаем число тегов.

In [5]:
original_inputs, original_targets = read_data('data/data.txt')

Проведём сортировку так, чтобы тег O имел позицию 0 в словаре

In [6]:
unique_tags = sorted(count_tags(original_targets).keys(), key=lambda tag: tag[2:])
experiment_config.n_classes = len(unique_tags)

In [7]:
print(unique_tags)

['O', 'B-company', 'I-company', 'B-facility', 'I-facility', 'B-geo-loc', 'I-geo-loc', 'B-movie', 'I-movie', 'B-musicartist', 'I-musicartist', 'B-other', 'I-other', 'B-person', 'I-person', 'B-product', 'I-product', 'B-sportsteam', 'I-sportsteam', 'B-tvshow', 'I-tvshow']


In [8]:
print(experiment_config.n_classes)

21


Препроцессинг токенов при помощи wordpiece токенайзера. Он применяется к каждому оригинальному токену по отдельности. Предполагается, что расщеплений хэштегов и имен не было. В случае если появились спец-токены (&lt;NUM&gt;, &lt;URL&gt;), не будем применять к ним токенайзер, но назначим им свободные слоты в словаре BERT ([unused1], [unused2]), это отображение описывается в словаре bert_specials. 

Функция также возвращает для каждого текста булеву маску, где True значения соответствуют местам, в которых подсчитываются предсказания для оригинальных токенов, как описано выше.

In [9]:
def bert_preprocessing(inputs, targets, bert_tokenizer, bert_specials=None):
    new_inputs = []
    new_targets = []
    masks = []
#     bert_tokenizer = pytorch_transformers.BertTokenizer.from_pretrained('bert-base-uncased')
    for text, tags in tqdm_notebook(zip(inputs, targets), total=len(inputs)):
        new_tokens, new_tags = [],[]
        mask = []
        for token, tag in zip(text, tags):
            if bert_specials and token in bert_specials:
                token_pieces = [bert_specials[token]]
            else:
                token_pieces = bert_tokenizer.tokenize(token)
            new_tokens.extend(token_pieces)
            new_tags.extend(split_tag(tag, len(token_pieces)))
            if experiment_config.mask_additional_wordpieces:
                mask.extend([True] + [False] * (len(token_pieces) - 1))
            else:
                mask.extend([True] * len(token_pieces))
        new_inputs.append(new_tokens)
        new_targets.append(new_tags)
        masks.append(mask)
    return new_inputs, new_targets, masks

Трансформация токенов, тегов и маски для модели Bert в индексы в словарях. Текст дополняется токенами [CLS] и [MASK], соответственно дополняются теги и маска.

In [10]:
def bert_numericalize_targets(targets, target_vocab):
    target_ids = target_vocab.numericalize(targets)
    return [[0] + ids + [0] for ids in target_ids]

def bert_numericalize_mask(masks):
    return [[False] + mask + [False] for mask in masks]

def bert_numericalize_inputs(inputs, bert_tokenizer):
    result = []
    for input_tokens in inputs:
        ids = [bert_tokenizer._convert_token_to_id(tok) for tok in input_tokens]
        ids = bert_tokenizer.add_special_tokens_single_sentence(ids)
        result.append(ids)
    return result

Объединим эти функции.

In [11]:
def bert_numericalize(inputs, targets, masks, bert_tokenizer, target_vocab):
    input_ids = bert_numericalize_inputs(inputs, bert_tokenizer)
    target_ids = bert_numericalize_targets(targets, target_vocab)
    mask_ids = bert_numericalize_mask(masks)
    return input_ids, target_ids, mask_ids

Теперь зададим Dataset и функцию создания мини-батча из тензоров. Батчи имеют динамический длину (макс. длина из выборки экземпляров)

In [12]:
class BertDataset(Dataset):
    def __init__(self, input_ids, target_ids, mask_ids):
        self.inputs = input_ids
        self.targets = target_ids
        self.masks = mask_ids
        assert len(input_ids) == len(target_ids) == len(self.masks)
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return (self.inputs[idx], self.targets[idx], self.masks[idx])
    
    @staticmethod
    def collate(examples):
        inputs, targets, masks = [],[],[]
        for inp, tgt, msk in examples:
            inputs.append(torch.tensor(inp, dtype=torch.long))
            targets.append(torch.tensor(tgt, dtype=torch.long))
            masks.append(torch.tensor(msk, dtype=torch.bool))
            
        input_tensor = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True)
        target_tensor = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)
        mask_tensor = torch.nn.utils.rnn.pad_sequence(masks, batch_first=True)
#         print(input_tensor.size(), target_tensor.size(), mask_tensor.size())
        return input_tensor, target_tensor, mask_tensor

In [13]:
bert_tokenizer = pytorch_transformers.BertTokenizer.from_pretrained('bert-base-cased')

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [14]:
bert_specials = {'<URL>': '[unused1]', '<NUM>': '[unused2]'}

Назначим тегам номера.

In [15]:
target_vocab = Vocab.from_id2word(unique_tags, unk_index=None, n_specials=0)

Проведём базовую предобработку (выбросим URL и заменим числа, если указано в конфигурации)

In [16]:
inputs, targets, _ = basic_preprocessing(original_inputs, original_targets,
                                      replace_urls=experiment_config.replace_urls,
                                      replace_numbers=experiment_config.replace_numbers,
                                      split_hashtags=False,
                                      split_mentions=False)

In [17]:
bert_tokenized_inputs, bert_tokenized_targets, bert_masks = bert_preprocessing(inputs, targets, bert_tokenizer, bert_specials)

Рассмотрим пример токенизации

In [18]:
print(original_inputs[0])

['Man', 'i', 'hate', 'when', 'people', 'carry', 'ragedy', 'luggage', '..', 'ima', 'just', 'rip', 'it', 'up', 'more', 'with', 'the', 'belt', 'loader', '#itaintmines']


In [19]:
print(bert_tokenized_inputs[0])

['Man', 'i', 'hate', 'when', 'people', 'carry', 'rage', '##dy', 'luggage', '.', '.', 'im', '##a', 'just', 'rip', 'it', 'up', 'more', 'with', 'the', 'belt', 'load', '##er', '#', 'it', '##ain', '##t', '##mine', '##s']


In [20]:
print(bert_tokenized_targets[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [21]:
print(bert_masks[0])

[True, True, True, True, True, True, True, False, True, True, False, True, False, True, True, True, True, True, True, True, True, True, False, True, False, False, False, False, False]


In [22]:
split_names = ['train', 'val', 'test']

Разобъём данные по предопределенным выборкам.

In [23]:
original_inputs_split, original_targets_split = split_to_dicts([original_inputs, original_targets],
                                                                 (train_indices, val_indices, test_indices), 
                                                                 split_names)

In [24]:
bert_inputs_split, bert_targets_split, bert_masks_split = split_to_dicts([bert_tokenized_inputs, bert_tokenized_targets, bert_masks],
                                                                 (train_indices, val_indices, test_indices), 
                                                                 split_names)

In [25]:
bert_numericalized_inputs, bert_numericalized_targets, bert_numericalized_masks = bert_numericalize(bert_tokenized_inputs, 
                                                                                                    bert_tokenized_targets,
                                                                                                    bert_masks, bert_tokenizer, target_vocab)

In [26]:
print(bert_numericalized_masks[0])

[False, True, True, True, True, True, True, True, False, True, True, False, True, False, True, True, True, True, True, True, True, True, True, False, True, False, False, False, False, False, False]


In [27]:
bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl = split_to_dicts([bert_numericalized_inputs, bert_numericalized_targets, bert_numericalized_masks],
                                                                            (train_indices, val_indices, test_indices), split_names)

Создадим соответствующие Datasetы.

In [28]:
train_dataset = BertDataset(*[spl['train'] for spl in (bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl)])

In [29]:
val_dataset = BertDataset(*[spl['val'] for spl in (bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl)])

In [30]:
test_dataset = BertDataset(*[spl['test'] for spl in (bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl)])

masked_crossentropy_loss подсчитывает loss по всему тензору, но при свёртке игнорирует позиции под маской.

In [31]:
def masked_crossentropy_loss(logits, targets, masked):
    loss_values = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), reduction='none')
    loss_values[masked.view(-1)] = 0
    return loss_values.sum() / (~masked).sum() # считаем среднее только по незамаскированным позициям

Используем претренированную реализацию от huggingface с "головой" для теггинга.
При forward-проходе сделаем маску, чтобы self-attention не работал по пэддингу.

In [32]:
class BERTForNER(nn.Module):
    def __init__(self, exp_config):
        super().__init__()
        self.model = pytorch_transformers.BertForTokenClassification.from_pretrained('bert-base-cased', 
                                                                                     num_labels=exp_config.n_classes)   
    def forward(self, inputs):
        attention_mask = (inputs != 0).type(torch.float32)
        return self.model(inputs, attention_mask=attention_mask)[0]

Опишем предсказание для оригинальной токенизации

In [33]:
def bert_compute_lengths_by_padding(inputs):
    return ((inputs != 0).sum(dim=-1) - 2).tolist()

def bert_predict_tags_with_mask(model, inputs, tag_mask, target_vocab):
    model.eval()
    result = []
    with torch.no_grad():
        logits = model(inputs)
        seqs = logits.argmax(dim=-1)
        for i,pred in enumerate(seqs):
            pred = pred[tag_mask[i]].tolist()
            result.append(target_vocab.transform_ids(pred))
    return result

def bert_predict_tags_for_loader(model, loader, target_vocab, use_mask=True, device='cuda'):        
    result = []
    for batch in loader:
        if use_mask:
            inputs,_,mask = [x.to(device) for x in batch]
            result.extend(bert_predict_tags_with_mask(model, inputs, mask, target_vocab))
        else:
            raise ValueError("Not implemented yet")
    return result

Опишем LightningModule, оборачивающий модель. В нём описывается поведение при обучении и валидации, а также загрузчики данных. Модуль полагается на глобальные переменные, но всё, что не касается обучения и текущего эксперимента, вынесено за его пределы и не полагается на них.

После каждого прохода по валидационной выборке, в лог записывается f1-метрика и выводится classification_report.

In [34]:
class LightningBERTMasking(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.model = BERTForNER(config)
        self.config = config
        self.lr = config.lr
        self.train_batch_size = config.train_batch_size
        self.val_batch_size = config.val_batch_size

    def forward(self, inputs):
        return self.model(inputs)
    
    def compute_loss_on_batch(self, batch):
        inputs, targets, mask = batch
        logits = self(inputs)
        
        loss_mask = ~mask
        loss = masked_crossentropy_loss(logits, targets, loss_mask)
        return loss
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        loss = self.compute_loss_on_batch(batch)
        return {'loss': loss}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        loss = self.compute_loss_on_batch(batch)
        inputs, targets, mask = batch
        predicted_tags = bert_predict_tags_with_mask(self, inputs, mask, target_vocab)
        
        return {'val_loss': loss, 'tags': predicted_tags}

    def validation_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.tensor([x['val_loss'] for x in outputs]).mean()
        predictions = list(chain.from_iterable(x['tags'] for x in outputs))
        f1_score = seqeval.metrics.f1_score(original_targets_split['val'], predictions)
        
        print(seqeval.metrics.classification_report(original_targets_split['val'], predictions))
        metrics = {'avg_val_loss': avg_loss.item(), 'f1': f1_score}
        metrics_to_write = dict(metrics, epoch=self.trainer.current_epoch+1)
#         metrics.update(self.trainer.tng_tqdm_dic)
        
#         scalar_metrics = self.trainer.__metrics_to_scalars(
#                     metrics, blacklist=self.trainer.__log_vals_blacklist())
        
        assert self.experiment
        self.experiment.log(metrics_to_write)
        self.experiment.save()
        
        return metrics

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    @pl.data_loader
    def tng_dataloader(self):
        # REQUIRED
        assert isinstance(train_dataset, BertDataset)
        return DataLoader(train_dataset, batch_size=self.train_batch_size, shuffle=True, collate_fn=BertDataset.collate)

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        assert isinstance(val_dataset, BertDataset)
        return DataLoader(val_dataset, batch_size=self.val_batch_size, shuffle=False, collate_fn=BertDataset.collate)

Проверим работоспособность модели (на CPU)

In [35]:
def test_bert_predict():
    bert = BERTForNER(experiment_config)
    dl = DataLoader(train_dataset, batch_size=4, collate_fn=BertDataset.collate)
    bert.eval()
    with torch.no_grad():
        for batch in dl:
    #         print(batch)
            res = bert_predict_tags_with_mask(bert, batch[0], batch[2], target_vocab)
            print(res)
            break

test_bert_predict()

[['B-tvshow', 'B-company', 'I-facility', 'I-product', 'B-product', 'I-musicartist', 'B-product', 'B-product', 'B-company', 'O', 'B-product', 'O', 'B-movie', 'B-company', 'O'], ['I-geo-loc', 'B-product', 'B-facility', 'O', 'B-facility', 'B-product', 'O', 'O', 'B-movie'], ['I-tvshow', 'I-product', 'B-product', 'O', 'B-facility', 'O', 'I-musicartist', 'B-musicartist', 'I-product', 'O', 'B-product', 'O', 'O', 'O', 'O', 'O', 'I-facility', 'B-movie', 'O', 'B-product', 'O', 'B-product'], ['I-geo-loc', 'B-product', 'B-facility', 'O', 'B-facility', 'B-facility', 'I-tvshow', 'B-product', 'B-facility', 'B-product', 'O', 'B-facility', 'B-facility', 'B-product', 'B-facility', 'B-product', 'B-facility', 'I-tvshow', 'O']]


## Запуск эксперимента


In [36]:
# import gc
# del pl_bert
# gc.collect()
# torch.cuda.empty_cache()

In [37]:
!nvidia-smi

Wed Sep  4 14:26:44 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 27%   38C    P8    10W / 250W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [38]:
exp = Experiment(save_dir=experiment_dir, name=experiment_config.experiment_name)

In [39]:
print(exp.version)

6


Сохраним конфигурацию эксперимента

In [40]:
exp.argparse(experiment_config)
exp.save()

Опишем место для сохранения чекпоинтов и критерий отбора (средний f1 по тегам) и ранней остановки.

In [41]:
checkpoint_path = f'{experiment_dir}/{experiment_config.experiment_name}/version_{exp.version}/checkpoint'

In [42]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath= checkpoint_path,
    save_best_only=True,
    verbose=True,
    monitor='f1',
    mode='max'
)

early_stop = pl.callbacks.EarlyStopping(
        monitor='f1',
        patience=5,
        verbose=True,
        mode='max'
)

Создадим модель и обучим её.

In [43]:
pl_bert = LightningBERTMasking(experiment_config)

In [44]:
print(len(pl_bert.tng_dataloader))

325


In [45]:
print(len(pl_bert.val_dataloader))

36


In [46]:
trainer = pl.Trainer(experiment=exp,
                     max_nb_epochs=experiment_config.n_epochs,
                     gpus=[0],
                     gradient_clip=experiment_config.gradient_clipping_norm,
                     early_stop_callback=early_stop,
                     accumulate_grad_batches=experiment_config.gradient_acccumulation_steps,
                     add_log_row_interval=100,
                     checkpoint_callback=checkpoint_callback)

VISIBLE GPUS: '0'
gpu available: True, used: True


In [47]:
trainer.fit(pl_bert)

  0%|          | 0/5 [00:00<?, ?it/s]

                                            Name                        Type  \
0                                          model                  BERTForNER   
1                                    model.model  BertForTokenClassification   
2                               model.model.bert                   BertModel   
3                    model.model.bert.embeddings              BertEmbeddings   
4    model.model.bert.embeddings.word_embeddings                   Embedding   
..                                           ...                         ...   
215                      model.model.bert.pooler                  BertPooler   
216                model.model.bert.pooler.dense                      Linear   
217           model.model.bert.pooler.activation                        Tanh   
218                          model.model.dropout                     Dropout   
219                       model.model.classifier                      Linear   

        Params  
0    108326421  
1    

  0%|          | 1/361 [00:00<01:36,  3.74it/s, batch_nb=0, epoch=0, gpu=0, loss=2.956, v_nb=6]

             precision    recall  f1-score   support

     tvshow       0.00      0.00      0.00        11
   facility       0.00      0.00      0.00        58
     person       0.01      0.02      0.02       131
    product       0.00      0.00      0.00        38
        loc       0.01      0.03      0.02       156
    company       0.01      0.02      0.01        84
      other       0.00      0.00      0.00       110
      movie       0.00      0.00      0.00        17
musicartist       0.00      0.00      0.00        33
 sportsteam       0.00      0.00      0.00        21

  micro avg       0.00      0.01      0.01       659
  macro avg       0.01      0.01      0.01       659



100%|██████████| 361/361 [00:53<00:00, 11.88it/s, avg_val_loss=0.187, batch_nb=324, epoch=0, f1=0.501, gpu=0, loss=0.206, v_nb=6]

...

  0%|          | 1/361 [00:00<00:30, 11.81it/s, avg_val_loss=0.185, batch_nb=0, epoch=4, f1=0.595, gpu=0, loss=0.056, v_nb=6]    

             precision    recall  f1-score   support

     tvshow       0.16      0.27      0.20        11
   facility       0.61      0.76      0.68        58
     person       0.67      0.79      0.73       131
    product       0.23      0.39      0.29        38
        loc       0.71      0.78      0.75       156
    company       0.54      0.76      0.63        84
      other       0.42      0.53      0.47       110
      movie       0.30      0.35      0.32        17
musicartist       0.32      0.39      0.35        33
 sportsteam       0.50      0.62      0.55        21

  micro avg       0.54      0.67      0.59       659
  macro avg       0.55      0.67      0.60       659

save callback...

Epoch 00004: f1 did not improve


100%|██████████| 361/361 [00:52<00:00, 12.03it/s, avg_val_loss=0.175, batch_nb=324, epoch=4, f1=0.626, gpu=0, loss=0.036, v_nb=6]

             precision    recall  f1-score   support

     tvshow       0.00      0.00      0.00        11
   facility       0.56      0.69      0.62        58
     person       0.76      0.79      0.77       131
    product       0.24      0.26      0.25        38
        loc       0.66      0.80      0.72       156
    company       0.71      0.62      0.66        84
      other       0.48      0.58      0.53       110
      movie       0.36      0.59      0.44        17
musicartist       0.55      0.48      0.52        33
 sportsteam       0.65      0.62      0.63        21

  micro avg       0.60      0.66      0.63       659
  macro avg       0.60      0.66      0.62       659

save callback...

Epoch 00005: f1 improved from 0.60857 to 0.62617, saving model to NER_experiments//BERT_finetune_mask/version_6/checkpoint/_ckpt_epoch_5.ckpt


1

### Inference

В этой части производятся предсказания на тестовой выборке. Метрики записываются в файл test_report.txt. Производится демонстрационное предсказание на свежем тексте.

In [48]:
import os

In [49]:
checkpoint_file = os.listdir(checkpoint_path)[0]
assert checkpoint_file.startswith('_ckpt_')
checkpoint_file_path = checkpoint_path + '/' + checkpoint_file
print(checkpoint_file_path)

NER_experiments//BERT_finetune_mask/version_6/checkpoint/_ckpt_epoch_5.ckpt


In [50]:
tags_path = f'{experiment_dir}/{experiment_config.experiment_name}/version_{exp.version}/meta_tags.csv'
print(tags_path)

NER_experiments//BERT_finetune_mask/version_6/meta_tags.csv


Загружаем чекпоинт

In [51]:
device = 'cuda'

In [52]:
pl_bert = LightningBERTMasking.load_from_metrics(checkpoint_file_path, tags_path, on_gpu=False)
pl_bert.freeze()
pl_bert = pl_bert.to(device)

In [53]:
test_predictions = bert_predict_tags_for_loader(pl_bert, DataLoader(test_dataset, collate_fn=BertDataset.collate, batch_size=24), target_vocab)

In [54]:
report = seqeval.metrics.classification_report(original_targets_split['test'], test_predictions, digits=3)

In [55]:
with open(f'{experiment_dir}/{experiment_config.experiment_name}/version_{exp.version}/test_report.txt', 'w+') as of:
    of.write(report)

In [56]:
def bert_predict_for_tokens(model, tokens, device='cuda'):
    fake_targets = [['O' for _ in text] for text in tokens]
    preproc_tokens, preproc_targets, _ = basic_preprocessing(tokens, fake_targets, 
                                                             replace_urls=experiment_config.replace_urls,
                                                              replace_numbers=experiment_config.replace_numbers,
                                                              split_hashtags=experiment_config.split_hashtags,
                                                              split_mentions=experiment_config.split_nicknames)
    preproc_tokens, preproc_targets, preproc_mask = bert_preprocessing(preproc_tokens, preproc_targets, bert_tokenizer, bert_specials)
    input_ids, target_ids, mask_ids = bert_numericalize(preproc_tokens, preproc_targets, preproc_mask, bert_tokenizer, target_vocab)
    model.eval()
    batch = BertDataset.collate(zip(input_ids, target_ids, mask_ids))
    inputs,_,mask = batch
    with torch.no_grad():
        result = bert_predict_tags_with_mask(model, inputs.to(device), mask.to(device), target_vocab)
    return result


Предскажем теги на отдельный пример

In [57]:
text_example = "Satellite imagery this morning of now Category 5 Hurricane Dorian approaching the Abaco Islands in the northern Bahamas. For the latest on Dorian visit http://hurricanes.gov"

In [58]:
import nltk

In [59]:
print(text_example)

Satellite imagery this morning of now Category 5 Hurricane Dorian approaching the Abaco Islands in the northern Bahamas. For the latest on Dorian visit http://hurricanes.gov


In [60]:
tokens_example = nltk.tokenize.TweetTokenizer().tokenize(text_example)
print(tokens_example)

['Satellite', 'imagery', 'this', 'morning', 'of', 'now', 'Category', '5', 'Hurricane', 'Dorian', 'approaching', 'the', 'Abaco', 'Islands', 'in', 'the', 'northern', 'Bahamas', '.', 'For', 'the', 'latest', 'on', 'Dorian', 'visit', 'http://hurricanes.gov']


In [61]:
prediction = bert_predict_for_tokens(pl_bert, [tokens_example])[0]

In [62]:
for token, tag in zip(tokens_example, prediction):
    print(token, tag)

Satellite O
imagery O
this O
morning O
of O
now O
Category O
5 O
Hurricane O
Dorian B-person
approaching O
the O
Abaco B-geo-loc
Islands I-geo-loc
in O
the O
northern O
Bahamas I-geo-loc
. O
For O
the O
latest O
on O
Dorian B-person
visit O
http://hurricanes.gov O


### Итоги:
В настоящий момент при помощи BERT получилось добиться f1 около .58 на тестовой выборке и .64 на валидационной (f1 на ней - критерий для остановки, так что стоит относиться к этому результату скептически). <br>
Precision и Recall в среднем близки, что не наблюдалось на некоторых простых моделях. <br>
Имеются трудности с распознаванием редких сущностей. Не были опробованы схемы со взвешиванием классов. Модель быстро переобучается (хотя часто f1 растет при увеличении val_loss), возможно, имеет смысл заморозить её части или использовать только её эмбеддинги, повысить уровень регуляризации в "голове".